## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Souillac,MU,-20.5167,57.5167,73.78,94,75,11.50,broken clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,59.88,76,0,4.61,clear sky
2,2,Magaria,NE,12.9983,8.9099,91.42,10,98,8.72,overcast clouds
3,3,Vestmannaeyjar,IS,63.4427,-20.2734,45.50,86,67,13.02,broken clouds
4,4,Esperance,AU,-33.8667,121.9000,67.84,42,99,17.27,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Kapaa,US,22.0752,-159.3190,78.78,79,75,17.27,broken clouds
15,15,Faanui,PF,-16.4833,-151.7500,81.64,72,11,10.63,few clouds
17,17,Chalchihuites,MX,23.4833,-103.8833,82.17,7,11,11.92,few clouds
20,20,Ahuimanu,US,21.4447,-157.8378,82.11,65,100,17.27,overcast clouds
24,24,Axim,GH,4.8699,-2.2405,79.56,87,38,5.59,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
648,648,Luanda,AO,-8.8368,13.2343,77.00,88,40,9.22,scattered clouds
663,663,Dwarka,IN,22.2394,68.9678,80.73,70,0,11.14,clear sky
666,666,Kinablangan,PH,7.6947,126.5503,77.29,92,100,5.03,light rain
668,668,Kupang,ID,-10.1667,123.5833,76.80,73,100,3.44,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                191
City                   191
Country                191
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Current Description    191
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
15,Faanui,PF,81.64,few clouds,-16.4833,-151.7500,
17,Chalchihuites,MX,82.17,few clouds,23.4833,-103.8833,
20,Ahuimanu,US,82.11,overcast clouds,21.4447,-157.8378,
24,Axim,GH,79.56,scattered clouds,4.8699,-2.2405,
28,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,
30,Avarua,CK,84.25,overcast clouds,-21.2078,-159.7750,
31,Cabedelo,BR,79.47,scattered clouds,-6.9811,-34.8339,
32,Manzanillo,CU,77.63,moderate rain,20.3433,-77.1167,
35,Dingle,PH,80.49,broken clouds,10.9995,122.6711,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Faanui,PF,81.64,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Chalchihuites,MX,82.17,few clouds,23.4833,-103.8833,Hotel chalchihuites
20,Ahuimanu,US,82.11,overcast clouds,21.4447,-157.8378,Paradise Bay Resort
24,Axim,GH,79.56,scattered clouds,4.8699,-2.2405,Axim Beach Resort And Spa
...,...,...,...,...,...,...,...
648,Luanda,AO,77.00,scattered clouds,-8.8368,13.2343,EPIC SANA Luanda
663,Dwarka,IN,80.73,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
666,Kinablangan,PH,77.29,light rain,7.6947,126.5503,Morales Eatery and Lodging House
668,Kupang,ID,76.80,overcast clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi


In [35]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))